In [9]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import imutils
from PIL import Image
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader

In [10]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using GPU:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("GPU not available, using CPU instead.")

Using GPU: NVIDIA GeForce RTX 2070 SUPER


In [11]:
labels = ['glioma', 'meningioma', 'notumor', 'pituitary']

x_train = [] # training images.
y_train  = [] # training labels.
x_test = [] # testing images.
y_test = [] # testing labels.

image_size = 150

for label in labels:
    trainPath = os.path.join('raw_data/Training',label)
    for file in tqdm(os.listdir(trainPath)):
        image = cv2.imread(os.path.join(trainPath, file), 0) # load images in gray.
        image = cv2.bilateralFilter(image, 2, 50, 50) # remove images noise.
        #image = cv2.applyColorMap(image, cv2.COLORMAP_BONE) # produce a pseudocolored image.
        image = cv2.resize(image, (image_size, image_size)) # resize images into 200*200.
        x_train.append(image)
        y_train.append(1 if label != 'notumor' else 0)
    
    testPath = os.path.join('raw_data/Testing',label)
    for file in tqdm(os.listdir(testPath)):
        image = cv2.imread(os.path.join(testPath, file), 0)
        image = cv2.bilateralFilter(image, 2, 50, 50)
        #image = cv2.applyColorMap(image, cv2.COLORMAP_BONE)
        image = cv2.resize(image, (image_size, image_size))
        x_test.append(image)
        y_test.append(1 if label != 'notumor' else 0)


x_train = np.array(x_train) / 255.0 # normalize Images into range 0 to 1.
x_test = np.array(x_test) / 255.0

print(x_train.shape)
print(x_test.shape)

100%|██████████| 300/300 [00:00<00:00, 404.02it/s]


(5712, 150, 150)
(1311, 150, 150)


In [12]:
# transform = transforms.Compose([
#     transforms.Resize((150, 150)),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485], std=[0.229])  # Adjust these values based on your dataset
# ])

In [13]:
# Convert numpy arrays to PyTorch tensors
train_images_tensor = torch.tensor(x_train).float() / 255.0  # Normalize if not already done
train_labels_tensor = torch.tensor(np.array(y_train)).long()  # Ensure labels are in long format for CE Loss

test_images_tensor = torch.tensor(x_test).float() / 255.0
test_labels_tensor = torch.tensor(np.array(y_test)).long()

# Reshape the tensors to add a channel dimension (1, 150, 150)
train_images_tensor = train_images_tensor.unsqueeze(1)
test_images_tensor = test_images_tensor.unsqueeze(1)

# Create TensorDatasets
train_dataset = TensorDataset(train_images_tensor, train_labels_tensor)
test_dataset = TensorDataset(test_images_tensor, test_labels_tensor)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, pin_memory=True, num_workers=3)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, pin_memory=True, num_workers=3)

In [14]:
class BrainMRINet(nn.Module):
    def __init__(self):
        super(BrainMRINet, self).__init__()
        # in_channels = the number of channels of the input to the convolutional layer (greyscale = 1, rgb = 3)
        # out_channels = the number of feature maps 
        # padding = (kernel_size-1) / 2
        # Use Conv2D since our image data is 2D.
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1)  # Change input channels if your images are not grayscale
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * 75 * 75, 128)  # Adjust the size here based on the output of your last pooling layer
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(F.relu(self.conv3(x)), 2)
        x = x.view(x.size(0), -1)  # Flatten the output
        x = F.relu(self.fc1(x))
        x = F.sigmoid(self.fc2(x))
        return x

model = BrainMRINet().to(device)

In [15]:
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [17]:
def train_model(num_epochs):
    model.train()  # Set the model to training mode
    
    for epoch in range(num_epochs):
        for images, labels in tqdm(train_loader):
            images, labels = images.to(device), labels.float().unsqueeze(1).to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')
        evaluate_model()

def evaluate_model():
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.float().unsqueeze(1).to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f'Test Accuracy: {accuracy}%\n')


train_model(50)
# evaluate_model()

100%|██████████| 90/90 [00:19<00:00,  4.63it/s]

Epoch 1/50, Loss: 0.01213163137435913


Test Accuracy: 1977.116704805492%



100%|██████████| 90/90 [00:16<00:00,  5.45it/s]

Epoch 2/50, Loss: 0.36424708366394043


Test Accuracy: 1977.116704805492%



 84%|████████▍ | 76/90 [00:14<00:02,  5.23it/s]


KeyboardInterrupt: 